# Weeks 12-13: Object localization

## General instructions

Every two weeks you will be given an assignment related to the associated module. You have roughly two weeks to complete and submit each of them. There are three weekly group sessions available to help you complete the assignments. Attendance is not mandatory but recommended. However, all assignments are graded and not submitting them or submitting them after the deadline will give you no points.

**FORMAT**: Jupyter notebook **(single file, not in a zip please!)**

**DEADLINE**: Sunday 11th April, 23:59

## Introduction

The objective of this assignment is to get a basic understanding of the core concepts in object detection using convolutional neural networks. More specifically, this work focuses on the simpler problem of object localization, where each image contains a single object that should be spatially localized and classified. While object localization is simpler than object detection in the sense that object detection is more general as it allows for multiple objects per scene, object localization remains a rather avanced method, perfectly suited for our current level. This exercise consists of two parts:
- To begin with, we will get familiar with the sliding window algorithm, and notice that applying a convolutional neural network sequentially to a specific set of windows within an image is in fact mathematically equivalent to passing the whole image as input for the convolutional neural network. We will in particular try to understand what parameters determine which set of windows will be processed by the convolutional sliding window algorithm and we will run a performance comparison between the sequential and convolutional implementations.
- In a second part, we will implement in pytorch our own take on the object localization algorithm. For that purpose, the knowledge acquired in the first part of this assignment will be helpful, as we will try to implement object localization using a convolutional sliding window approach. In order for us to be able to train and test our network even on modest hardware, this exercise will have you work on a custom dataset: you will use the MNIST dataset as a backbone to generate new images with digits randomly placed and transformed, and each image will contain a single digit. While solving object localization on this custom dataset will be much easier than traditional object detection on large datasets such as COCO, it will still require clever vectorized coding if you hope to train your network in a reasonable amount of time !

## Andrew's Videos related to this week's assignment

- [C4W3L01 Object Localization](https://www.youtube.com/watch?v=GSwYGkTfOKk&list=PL_IHmaMAvkVxdDOBRg2CbcJBq9SY7ZUvs)
- [C4W3L03 Object Detection](https://www.youtube.com/watch?v=5e5pjeojznk&list=PL_IHmaMAvkVxdDOBRg2CbcJBq9SY7ZUvs&index=3)
- [C4W3L04 Convolutional Implementation Sliding Window](https://www.youtube.com/watch?v=XdsmlBGOK-k&list=PL_IHmaMAvkVxdDOBRg2CbcJBq9SY7ZUvs&index=4)

# Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm

torch.backends.cudnn.benchmark = True

# Convolutional sliding window

The sliding window algorithm refers to any procedure that applies a certain function ("map") to sliding windows within an image. In its typical forumaltion, this seems to be an inherently sequential algorithm: process the first window, then the second etc$\dots$ assuming the function to apply on every window is a convolutional neural network, an interesting fact arises: the sequential implemention of sliding window for this convolutional neural network map becomes exactly equivalent to passing the whole image as input to the convolutional neural network. We will investigate this fact further in this section. Answer the following questions:

1) Watch Andrew's video about convolutional sliding window implementation.

2) What slight modification is needed on the convolutional neural network such that sequential and convolutional sliding window procedures for this convolutional neural network map become equivalent ?
    - The fully connected layers need to be 1x1xX convolutional layers instead.
3) Reproduce the convolutional neural network from Andrew's video (1:20), and verify that for an input tensor with spatial dimension $16\times 16$, convolutional sliding window for that convolutional neural network map is the same as a sequential sliding window with stride $2$ along both spatial dimension. You can for instance compute the norm of the difference between the output tensors from convolutional and sequential implementations and check that this norm is indeed $0.0$.

In [18]:
class SeqSliding(nn.Module):

    def __init__(self):
        super().__init__()
        self.stride = 2
        self.layers = []
        self.layers.append(nn.Conv2d(3, 16, (5, 5)))
        self.layers.append(nn.MaxPool2d((2,2), stride=2))
        self.layers.append(nn.Flatten())
        self.layers.append(nn.Linear(400, 400))
        self.layers.append(nn.Linear(400, 4))
        self.layers.append(nn.Softmax(dim=1))


    def forward(self, x):
        a = x
        a = self.split_to_windows(a)
        for layer in self.layers:
            a = layer(a)
        a = torch.reshape(a, (x.shape[0], a.shape[0]//x.shape[0], a.shape[1]))
        return a

    def split_to_windows(self, a):
        dirs = [(0, 0), (1, 0), (0, 1), (1, 1)]
        dims = ((a.shape[2] - 14)// self.stride) * 4 * a.shape[0]
        new_a = torch.empty((dims, 3, 14, 14))
        idx = 0
        for i in range(a.shape[0]):
            step = 1
            while step * self.stride <= a.shape[2] - 14:
                for x, y in dirs:
                    new_a[idx] = torch.narrow(torch.narrow(a[i], -2, y*self.stride, 14), -1, x*self.stride, 14)
                    idx += 1
                step += 1
        print(new_a[0].shape)
        print(new_a[0])
        return new_a



class ConvSliding(nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = []
        self.layers.append(nn.Conv2d(3, 16, (5, 5)))
        self.layers.append(nn.MaxPool2d((2,2), stride=2))
        self.layers.append(nn.Conv2d(16, 400, (5, 5)))
        self.layers.append(nn.Conv2d(400, 400, (1, 1)))
        self.layers.append(nn.Conv2d(400, 4, (1, 1)))



    def forward(self, x):
        a = x
        for layer in self.layers:
            a = layer(a)
        return a

In [3]:
x1 = torch.rand((2, 3, 14, 14))
t1 = torch.rand((2, 3, 16, 16))

In [19]:
conv_sliding = ConvSliding()
window_sliding = SeqSliding()
print(t1[0].shape)
print(t1[0])

oc2 = conv_sliding(t1)
# print(oc2)
wc2 = window_sliding(t1)
# print(wc2)

torch.Size([3, 16, 16])
tensor([[[0.5214, 0.4446, 0.5186, 0.2675, 0.7272, 0.2370, 0.8648, 0.7553,
          0.7156, 0.4497, 0.9696, 0.7582, 0.2733, 0.4350, 0.8688, 0.5231],
         [0.7472, 0.6933, 0.5925, 0.3003, 0.3372, 0.5689, 0.5460, 0.7994,
          0.8069, 0.9932, 0.5746, 0.3974, 0.9993, 0.8036, 0.1133, 0.5034],
         [0.0579, 0.0433, 0.4593, 0.7174, 0.2194, 0.8610, 0.0635, 0.0479,
          0.2487, 0.6071, 0.9336, 0.4425, 0.4596, 0.1362, 0.6895, 0.9062],
         [0.1829, 0.1440, 0.8905, 0.2512, 0.2231, 0.4686, 0.6640, 0.6587,
          0.6468, 0.2969, 0.3987, 0.9883, 0.4199, 0.7333, 0.0995, 0.2555],
         [0.2971, 0.9746, 0.8697, 0.2272, 0.7356, 0.6824, 0.0795, 0.1394,
          0.5140, 0.7101, 0.1662, 0.5048, 0.0279, 0.6754, 0.9259, 0.6877],
         [0.6254, 0.8872, 0.0365, 0.4608, 0.7303, 0.6824, 0.3050, 0.9470,
          0.1921, 0.6565, 0.4533, 0.9384, 0.3694, 0.9202, 0.1096, 0.5361],
         [0.6971, 0.8368, 0.6107, 0.4196, 0.9492, 0.6077, 0.8653, 0.5146,
        

In [11]:
print(oc2.shape, wc2[0].shape)
n = torch.reshape(oc2[0], (4,4))-wc2[0]
print(torch.linalg.norm(n))

torch.Size([2, 4, 2, 2]) torch.Size([4, 4])
tensor(1.0818, grad_fn=<CopyBackwards>)


4) What happens when the input tensor now has spatial dimensions $28\times 28$? Try a few other spatial dimensions. Can you find some such that convolutional and sequential procedures yield different outputs ?

5) Do a time comparison for various input spatial dimensions between convolutional and sequential sliding window procedures for Andrew's convolutional network map. Which is the most efficient ? Can you explain why ?

6) Modify slightly Andrew's convolutional neural network such that there is now an additional convolutional layer. Do convolutional and sequential sliding window with stride $2$ procedures still coincide ? If not, what should be the stride for them to coincide again ?

7) Same question but now add an additional max pooling layer (with kernel size $2$) instead.

8) It turns out that the architecture of the convolutional neural network map itself will determine what is the set of windows for which convolutional and sequential sliding window procedures coincide for that convolutional neural network map. In the following, let us assume that all convolutional layers have padding $0$ and all max pooling layers have kernel size $2$. Can you guess the formulas that give the number of windows and their indices ? $\textbf{Hint}$: Remember your implementation of the Conv2d layer. $\textbf{If you can't find this, CONTACT US!}$ It will be important in the second section.

# MNIST localization in Pytorch

## 1 Data generation

In this second section we aim to implement from scratch our custom digit localization algorithm. We will use the MNIST dataset as a backbone:

In [ ]:
def load_MNIST(MNIST_path, preprocessor, num_workers):
    train_dataset = datasets.MNIST(MNIST_path, train=True, transform=preprocessor)
    val_test_dataset = datasets.MNIST(MNIST_path, train=False, transform=preprocessor)
    n_val_test = len(val_test_dataset.targets)
    # The test set will only contain 30 images, perfect for visualization purpose:
    val_dataset, test_dataset = torch.utils.data.random_split(val_test_dataset, [n_val_test-30, 30])
    MNIST_datasets = {"train": train_dataset, 
                      "val": val_dataset,
                      "test": test_dataset}
    MNIST_generators = {"train": torch.utils.data.DataLoader(MNIST_datasets["train"], 
                                                             batch_size=4, 
                                                             shuffle=True, 
                                                             num_workers=num_workers),
                        "val": torch.utils.data.DataLoader(MNIST_datasets["val"], 
                                                           batch_size=256, 
                                                           shuffle=False, 
                                                           num_workers=num_workers),
                        "test": torch.utils.data.DataLoader(MNIST_datasets["test"], 
                                                            batch_size=30, 
                                                            shuffle=False, 
                                                            num_workers=num_workers)}
    return MNIST_datasets, MNIST_generators

We now want to have a digit generator in pytorch that would produce samples "on the fly" for the digit localization task. The problem we have is that all MNIST digits are centered, have the same orientation and roughly the same size which makes the localization task pretty dull. 

**1) Write a data preprocessor that would do the following:**

**a) Randomly rotates digits.**

**b) Randomly rescales digits with a scale factor randomly sampled in the range [0.75,1.25].**

**c) Randomly places digits within a zeros tensor having spatial dimensions $64\times 64$.**

**You can make sure your data generator works as intended by visualizing the test set generated by MNIST_generators["test"]. Each image generated this way should contain one and exactly one digit!**

In [ ]:
MNIST_path = '../Data/MNIST'
preprocessor = [TODO]
num_workers = 2
MNIST_datasets, MNIST_generators = load_MNIST(MNIST_path, preprocessor, num_workers)

## 2 Convolutional sliding window

We then need to implement the convolutional neural network that will do the heavy work. In a nutshell, we want to have a CNN that is able to locate and classify digits within input images with base dimensions $(h\_in, w\_in)$. Then, we use this CNN as the backbone mapping in a sliding window procedure, in order to be able to locate and classify digits within larger images with dimensions $(H\_in, W\_in)$ such that $H\_in > h\_in$ and $W\_in > w\_in$. In the first section, you learned how to perform this efficiently by converting the sequential sliding window procedure into a convolutional one. More specifically, you learned that passing directly the whole large image to the CNN mapping gives an output tensor whose spatial dimensions $(H\_out, W\_out)$ correspond to the number of sliding windows in the sequential procedure such that each cell is exactly the output of the CNN applied to the corresponding window. Moreover, you also are able to find the indices of these windows. 

**2) Use this knowledge in order to implement the CNN you'll be using for the digit localization task. Remember that the window size $(h\_in, w\_in)$ should be large enough to contain digits entirely and the input size $(H\_in, W\_in)$ should match the dimensions of the tensors produced by you generator, namely $64\times 64$.**

## 3 Bounding box extraction, encoding

At the moment, we only have access to the digit labels. This is not sufficient for the task of digit localization: we need those bounding boxes, so we will produce them ourselves ! 

**3-1) Implement a code that will produce sharp bounding boxes around a digit. You can for instance use torch.where(... > 0.0) to find the limits of the digit in an image.**

We will encode the digit localization problem using a target vector of size $6$: $y = (y_0, y_1, y_2, y_3, y_4, y_5)$:
- $y_0$ is binary and encodes whether or not a digit was detected.
- $y_1$ is categorical and encodes the label of the digit (if any).
- $y_2, y_3, y_4, y_5$ are continuous and encode the bounding box of the digit (if any).

We want every window to carry information. As such, we want to produce one $y$ for every window, such that the groudtruth tensor $Y\_true$ should have dimensions $(N,6,H\_out,W\_out)$. In order to do this, for each window we can check if the digit is visible in this window:
- If that's the case, $y_0$ is $1$, $y_1$ is the label of the digit and $y_2, y_3, y_4, y_5$ are the bounding box values **converted in the referential of the window**: the top of the window $h\_start$ is treated as $0.0$ wheras the bottom of the window $h\_end$ is treated as $1.0$ (and similarly for the left and right of the window). 
- Otherwise, the $y$ for that window is simply a zeros vector (if that's the case, we only really care about $y_0=0$).

**3-2) Implement the encoding procedure. The code may have a similar structure as in the cell below:**

In [ ]:
H_out = [TODO]
W_out = [TODO]
bounding_boxes = torch.zeros(size=(N, 4))
Y_true = torch.zeros(size=(N, 6, H_out, W_out))
for n in range(N):
    # Bounding box extraction:
    bounding_boxes[n,:] = [TODO]
    # Bounding box coordinates:
    top = bounding_boxes[n,0]
    bottom = bounding_boxes[n,1]
    left = bounding_boxes[n,2]
    right = bounding_boxes[n,3]
    for h in range(H_out):
        for w in range(W_out):
            # Coordinates of the window (h,w):
            h_start = [TODO]
            w_start = [TODO]
            h_end = [TODO]
            w_end = [TODO]
            # If a digit is visible in the window (h,w) <==>
            # rectangles (top, bottom, left, right) and (h_start, h_end, w_start, w_end) intersect:
            if [TODO]:              
                Y_true[n,:,h,w] = [TODO]

## 4 Training loop

We are now ready to train our model. The only remaining question is the loss. We want a loss of the form $\sum_{n < N} \sum_{h < H\_out} \sum_{w < W\_out} L(Y\_true[n,:,h,w], Y\_out[n,:,h,w])$,
where 

$L(Y\_true[n,:,h,w], Y\_out[n,:,h,w]) = L\_detection + L\_classification + L\_regression$ such that

- $L\_detection = binary\_cross\_entropy(sigmoid(Y\_out[n,0,h,w]), Y\_true[n,0,h,w])$

If $sigmoid(Y\_out[n,0,h,w]) > 0.5$ and $Y\_true[n,0,h,w] == 1$ ("true positive detection of digit in window $(h,w)$"):
- $L\_classification = negative\_log\_likelihood\_loss(log\_softmax(Y\_out[n,1\!:\!11,h,w]), Y\_true[n,1,h,w])$ 
- $L\_regression = mse(Y\_out[n,-4\!:,h,w], Y\_true[n,-4\!:,h,w])$

Else:
- $L\_classification=0.0$
- $L\_regression=0.0$

Essentially, this loss indicates that we break down the whole task into three subproblems:

a) Is there a digit in the current window ?

If so,

b) What is the digit in the window ?

c) Where is the digit in the window ?

The two last questions only make sense assuming the first question was positively answered, hence the use of a conditional loss. Notice also that we sum the loss not only over samples in a batch, but also over windows. Essentially, the set of windows forming the sliding window procedure can be seen itself as a minibatch of samples. 

**4) Implement the training loop, then train your model for a few epochs.**

## 5 Prediction

Your model should now be trained, great ! But now you may wonder: "How do I predict ?". And indeed it is not so obvious. In our implementation, our model should output a tensor $Y\_out$ with dimensions $(N,15,H\_out,W\_out)$, that is we have one prediction vector per window in the sliding window procedure. We propose the following strategy: among all the predicted vectors, keep the one associated to the window with the highest label classification score. In other word, $\forall n<N$:

a) Compute the max classification scores: $max\_classification\_scores = max_{i=1:11} \{log\_softmax(Y\_out[n,i,:,:])\}$.

b) Extract the indices of the best window: $h\_best,w\_best = argmax_{h<H\_out,w<W\_out} \{max\_classification\_scores[h,w]\}$.

c) Save the prediction associated to the best window. **You will have to convert the predicted bounding box back to the referential of the input image.** 

**5) Implement the predict function.**

## 6 Visualization

Time to test your hard work by visualizing your predictions ! 

**6) Implement a function that will display the 30 images contained in the test set, with the true bounding boxes in green and the predicted bounding boxes in red overlayed over images. Each image will be titled with its groundtruth and predicted labels. You can use fig, axes = plt.subplots(6, 5, figsize=(20, 16)) in order to place the images conveniently.**